In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
 
import plotly.graph_objs as go
from textblob import TextBlob 
from statistics import *
import plotly.express as px
import string
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.style.use('ggplot')
from nltk.corpus import stopwords
 

In [ ]:
df=pd.read_csv('../input/all-trumps-twitter-insults-20152021/trump_insult_tweets_2014_to_2021.csv')

In [ ]:
df.head(10)

In [ ]:
df.shape

In [ ]:
df.info()

since there are no match missing data ,we will clearly remove just 2 samples

In [ ]:
df=df.dropna()

In [ ]:
#df.set_index('date',inplace=True)

In [ ]:
df['date']=pd.to_datetime(df.date,format='%Y-%m-%d')

In [ ]:
#number of trump tweets per year
df['date'].value_counts().resample('Y').sum()

In [ ]:
df['date'].value_counts().resample('Y').sum().plot.line()

In [ ]:
#number of trump tweets per month
df['date'].value_counts().resample('m').sum().plot.line()

In [ ]:
df.head(5)

In [ ]:
df['target'].unique()

In [ ]:
a=df['target'].value_counts().sort_values(ascending=False)
top10 = a.head(10)
top10

# let's analysis top 10 targets that trump interested in 

# First we will talk about media tweets 
    ** Doing some analysis 
    ** performing sentimental analysis

In [ ]:
top10_targets=df[df['target'].isin(top10.index)]
top10_targets


In [ ]:
a=top10_targets['insult'].value_counts()
#top ten insults in media tweets
b=a[:10]
px.bar(top10_targets, x=b.values,y= b.index, color=b.index, height=500)

In [ ]:
#find and count mentions


def mentions(text):
    line=re.findall(r'(?<=@)\w+',text)
    return " ".join(line)
top10_targets['mentions']=top10_targets['tweet'].apply(lambda x:mentions(x))

temp=top10_targets['mentions'].value_counts()[:][1:11]
 
px.bar(top10_targets, x=temp.values,y= temp.index, color=temp.index, height=500)

In [ ]:
#treemap of mentions
b = top10_targets['mentions'].value_counts()[:][1:11].index.tolist()
a = top10_targets['mentions'].value_counts()[:][1:11].tolist()
row = pd.DataFrame({'scenario' : []})
row["scenario"] = b
row["Percentage"] = a
fig = px.treemap(row, path= ["scenario"], values="Percentage",title='Tree of Mentions')
fig.show()


In [ ]:
#4. Number of words in a tweet

fig = plt.figure()
ax1 = plt.axes()

#fig,ax1=plot(figsize=(15,5))
#fig,ax1=plt.figure()
tweet_len=top10_targets['tweet'].str.split().map(lambda x: len(x))
ax1.hist(tweet_len,color='#17C37B')
ax1.set_title('top10_targets tweets')

In [ ]:

#count and find hashtags


def find_hash(text):
    line=re.findall(r'(?<=#)\w+',text)
    return " ".join(line)
top10_targets['hash']=top10_targets['tweet'].apply(lambda x:find_hash(x))

#find top ten hashtags in media tweets

temp=top10_targets['hash'].value_counts()[:][1:11]
px.bar(top10_targets, x=temp.values,y= temp.index, color=temp.index, height=500)

clean media tweets

In [ ]:
def clean_tweets(tweet):
 
    tweet = re.sub(r':', '', str(tweet))
    tweet = re.sub(r'‚Ä¶', '', str(tweet))
    tweet = re.sub('@[A-Za-z0–9]+', '', str(tweet))               
    tweet = re.sub('#', '', str(tweet)) # Removing '#' hash tag
    tweet = re.sub('https?:\/\/\S+', '', str(tweet)) # Removing hyperlink
    #replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ',str(tweet))
    tweet = str(tweet).lower()
    tweet = re.sub('\[.*?\]', '', tweet)
    tweet = re.sub('https?://\S+|www\.\S+', '', tweet)
    tweet = re.sub('<.*?>+', '', tweet)
    tweet = re.sub('\n', '', tweet)
    tweet = re.sub('\w*\d\w*', '', tweet)
    tweet = re.sub(r'#','',tweet)
    tweet = re.sub(r'RT[\s]+','',tweet)
    tweet = re.sub(r'[^\w]', ' ', tweet)
    tweet = re.sub('https', '', tweet)
    tweet = re.sub('https ', '', tweet)
    tweet = re.sub('co', '', tweet)
    words = [w for w in tweet if w not in stopwords.words('english')]

    return tweet

# Applying the cleaning function to both test and training datasets
top10_targets['tweet'] = top10_targets['tweet'].apply(lambda x: clean_tweets(x))

In [ ]:
from wordcloud import WordCloud
fig, ax1 = plt.subplots(1, 1, figsize=[26, 8])
wordcloud1 = WordCloud( background_color='white',
                        width=600,
                        height=400).generate(" ".join(top10_targets['tweet']))
ax1.imshow(wordcloud1)
ax1.axis('off')
ax1.set_title('top10_targets',fontsize=40);

# Doing sentimental analysis for media tweets

In [ ]:
## Sentimental Analysis 

def getSubjectivity(tweet):
    try:
        return TextBlob(tweet).sentiment.subjectivity
    except:
        return None

In [ ]:
# Create a function to get the polarity
def getPolarity(tweet):
    try:
        return  TextBlob(tweet).sentiment.polarity
    except:
        return None

def sentiment_calc(tweet):
    try:
        return TextBlob(tweet).sentiment
    except:
        return None

In [ ]:
# Create two new columns 'Subjectivity' & 'Polarity'
top10_targets['subjectivity'] = top10_targets['tweet'].apply(getSubjectivity)
top10_targets['polarity'] = top10_targets['tweet'].apply(getPolarity)

#calculating sentimental

top10_targets['sentiment'] = top10_targets['tweet'].apply(sentiment_calc)


In [ ]:
def getAnalysis(score):
    if score < 0:
      return 'Negative'
    elif score == 0:
      return 'Neutral'
    else:
      return 'Positive'

top10_targets['analysis'] = top10_targets['polarity'].apply(getAnalysis)

In [ ]:
top10_targets.groupby('analysis')['analysis'].count().plot.bar()
plt.title('Sentiment Analyisis for top10_targets tweets ')

we seeing that most of all trump tweets about media are negative 

In [ ]:
x=top10_targets.groupby('analysis')['analysis'].count()
neutral=top10_targets[top10_targets['analysis']=='Neutral']['tweet'].count()
positive=top10_targets[top10_targets['analysis']=='Positive']['tweet'].count()
negative=top10_targets[top10_targets['analysis']=='Negative']['tweet'].count()

fig = go.Figure(data=[go.Pie(labels=["positivity","negativity","neutrality"], values=[positive,negative,neutral])])
fig.update_layout(title_text='Sentiment analysis of top10_targets tweets')
fig.show()


we will do the same steps for ( democrats , hillary-clinton , trump-russia ,  joe-biden , the-new-york-times ,  cnn    ,   impeachment-inquiry  ,  adam-schiff    ,     2020-election) tweets  to do analysis and performing sentimental analysis 


if you  find benefits in this netbook please upvote me , and if you have any question you are welcome in comments  

In [ ]:
# top10_targets.groupby('target')['polarity'].values.plot.bar()
# plt.title('Sentiment Analyisis for top10_targets tweets')

px.bar(top10_targets, x=top10_targets['polarity'].values, y= top10_targets['target'].index, color=temp.index, height=500)